# Der Ruin des Spielers


In dieser Fallstudie studieren wir unsere Gewinnchancen beim Spielen im Casino. Insbesondere wollen wir am Ende folgende Frage beantworten können:

> Angenommen wir gehen mit einem bestimmten Anfangsbudget ins Casino und unser Ziel ist es, einen bestimmten Zielbetrag zu erreichen. Dabei setzen wir jeweils immer einen fixen Betrag. Was ist die Wahrscheinlichkeit, dass wir gewinnen (d.h. den Zielbetrag erreichen) und was ist die Wahrscheinlichkeit, dass wir dabei Bankrott gehen?

Dabei werden wir sehen, wie uns Funktionen und Prozeduren helfen, eine Aufgabe zu strukturieren. 
Zudem lernen wir auch, wie wir in einer Simulation Zufallszahlen nutzen können. 


### Zufallszahlen generieren

Jede Programmiersprache stellt uns Funktionalität zur Verfügung, um zufällige Zahlen zu generieren. Um zufällige Zahlen in Python zu generieren nutzen wir die Bibliothek ```random```, die wir aber zuerst importieren müssen. 

In [3]:
import random

Mithilfe der Funktion ```random.randint(a, b)``` können wir eine zufällige ganze Zahl im Interval $[a, b]$ generieren. 
Das Modul stellt auch viele weitere Funktionen zur Verfügung, die Sie in der [Dokumentation](https://python.readthedocs.io/en/stable/library/random.html) finden. Wir brauchen hier nur diese eine Funktion. 

In [ ]:
random.randint(0, 10)

Wir sehen, dass wenn wir diese Funktion mehrmals ausführen, jedes Mal ein anderer Wert erzeugt wird. Mittels einer Schleife können wir auf diese Art viele verschiedene zufällige Werte generieren. 

In [ ]:
# Ihre Aufgabe: Nutzen Sie eine Schleife um Zufallszahlen zu erzeugen.
# Variieren Sie auch die Argumente zu randint. Wie werden die Intervallgrenzen behandelt?

i = 0
while i < 20:
    print(random.randint(0,10))
    i = i + 1
    
# Die Intervallgrenzen können auch erzeugt werden.

### Ein Spiel simulieren

Nun nutzen wir diese Zufallsfunktion um ein [Roulettespiel](https://de.wikipedia.org/wiki/Roulette) zu simulieren. Wir schreiben uns dafür eine Funktion. Diese bekommt als Argument unseren Einsatz, und gibt unseren Gewinn zurück. Was wir zurückbekommen ist zwei mal der Einsatz im Gewinnfall, oder 0 falls wir nicht gewinnen. Wir setzen dabei immer nur auf eine der Farben und schliessen nie andere Wetten ab.
Da es keinen Unterschied für die Gewinnchancen macht ob wir auf Rot oder Schwarz setzen, bilden wir dies in unserer Funktion nicht ab. Wir implementieren nur die Tatsache, dass wir in 19 von 37 Fällen verlieren und in 18 gewinnen. 

In [6]:
def play_game(bet): 
    random_number = random.randint(0, 36)
    if random_number == 0:
        return 0
    elif random_number <= 18:
        return 0
    else: 
        return 2 * bet

In [ ]:
play_game(5)

### Der durchschnittliche Gewinn

Die vielleicht einfachste, aber dennoch interessante Frage ist, wieviel Geld wir im Durchschnitt gewinnen. Bei diesem einfachen Spiel könnten wir dies noch ohne Programm überprüfen. Bei komplexeren Spielen ist es dann aber oft einfacher, eine Simulation zu schreiben. Wir wollen nun `n` Spiele mit dem immer gleichen Einsatz `bet` simulieren.

In [8]:
def average_win_in_n_games(n, bet):
    total_win = 0
    i = 0
    
    while i < n:
        win = play_game(bet)
        total_win = total_win + win
        i = i + 1
        
    average_win = total_win / n
    return average_win

Wir können nun überprüfen, dass wenn ```n``` gross wird, wir immer einen Verlust einfahren werden. Dies ist der Gewinn des Casinos. Je grösser die Anzahl Spieler, desto stabiler der Gewinn. 

In [ ]:
average_win_in_n_games(100000, 100)

### Spielen mit Zielbetrag

Statt auf ein einziges Spiel fokussieren wir uns nun auf eine Serie von Spielen. Dabei verwenden wir die folgende Strategie:
- Wir spielen so lange weiter, bis wir einen bestimmten Zielbetrag erreicht haben.
- Wenn wir aber kein Geld mehr haben, müssen wir aufhören zu spielen.

Da wir schon die Funktion ```play_game``` geschrieben haben, können wir diese hier wiederverwenden. Wir nutzen diese Funktion als *black box*, d.h., wir müssen gar nicht wissen, wie die Funktion implementiert ist oder welches Spiel da eigentlich simuliert wird. Wichtig ist nur, dass wir wissen, wie wir die Funktion aufrufen und dass der Rückgabewert jeweils unser Gewinn oder Verlust ist.

Die folgende Funktion soll dann `True` zurück geben wenn wir das Ziel erreicht haben oder `False` wenn wir bankkrott sind.

In [ ]:
def play_until_target_reached_or_ruined(initial_amount, bet, target_amount):
    balance = initial_amount
    
    while balance >= bet and balance < target_amount:
        balance = balance - bet
        balance = balance + play_game(bet)

    return (balance >= target_amount)

In [ ]:
play_until_target_reached_or_ruined(100, 10, 200)

### Simulation - Was ist unsere Gewinnwahrscheinlichkeit?

Zum Schluss schreiben wir uns eine Funktion, die uns eine Anzahl Versuche simuliert und dabei zählt, wie oft wir unser Ziel erreichen und wie oft wir bankrott gehen. Auch hier nutzen wir nun wieder die zuvor definierte Funktion.

In [ ]:
def count_wins_in_n_games(number_of_tries, initial_amount, bet_per_game, target_amount):

    num_wins = 0

    i = 0
    while i < number_of_tries:
        if play_until_target_reached_or_ruined(initial_amount, bet_per_game, target_amount) == True:
            num_wins = num_wins + 1
        i += 1

    return num_wins

Um die Gewinnwahrscheinlichkeit zu berechnen, müssen wir nur nur diese Funktion aufrufen und den Anteil an Gewinnen berechnen.

In [ ]:
number_of_games = 1000
target_amount = 300
initial_amount = 200
bet = 10
number_of_wins = count_wins_in_n_games(number_of_games, initial_amount, bet, target_amount)

probability_to_win = number_of_wins / number_of_games
probability_to_lose = (number_of_games - number_of_wins) / number_of_games
expected_win = probability_to_win * (target_amount - initial_amount ) + probability_to_lose * (-initial_amount)
print("Wahrscheinlichkeit zu gewinnen: ", probability_to_win)
print("Wahrscheinlichkeit zu verlieren ", probability_to_lose)
print("Erwarteter Gewinn ", expected_win)
